In [8]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import requests
import re
import spacy
import webbrowser
import time
from tqdm import tqdm

# Load the spaCy model
nlp = spacy.load('en_core_web_md')

# Cooking tools list
cooking_tools = [
    "mixer", "knife", "spoon", "spatula", "whisk", "measuring cup", "frying pan",
    "baking sheet", "mixing bowl", "oven", "saucepan", "skillet", "blender",
    "grater", "rolling pin", "mortar and pestle", "colander", "strainer", "tongs",
    "peeler", "ladle", "cutting board", "pot", "pan", "grill", "oven mitt",
    'mortar and pestle', 'whisk', 'oven mitt', 'measuring cup', 'frying pan', 
    'oven', 'spoon', 'blender', 'strainer', 'knife', 'cutting board', 'spatula', 
    'tongs', 'rolling pin', 'grater', 'peeler', 'colander', 'mixing bowl', 
    'ladle', 'measuring spoons', 'microwave', 'toaster', 'baking sheet', 
    'slow cooker', 'salad spinner', 'zester', 'meat thermometer', 
    'cookie cutter', 'garlic press'
]

def progress_bar(task_name):
    for _ in tqdm(range(100), desc=task_name, bar_format="{l_bar}{bar} [🕒 {remaining}]"):
        time.sleep(0.01)

def get_driver():
    print("🌟 Setting up the web driver... Please wait.")
    progress_bar("Setting up driver")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()))

def get_soup(driver, url):
    print("🔗 Processing the URL... Please wait.")
    progress_bar("Processing URL")
    driver.get(url)
    return BeautifulSoup(driver.page_source, 'html.parser')

def get_title(soup):
    print("📋 Extracting the title... Please wait.")
    progress_bar("Extracting title")
    title_element = soup.find('h1', class_='heading__title') or soup.find('h1')
    if title_element is not None:
        title = title_element.text.strip()
        return f"✨ Recipe Title: {title} ✨"
    else:
        return "🚫 Title not found"

def get_ingredients(soup):
    print("🥕 Extracting the ingredients... Please wait.")
    progress_bar("Extracting ingredients")
    ingredient_sections = soup.find_all(lambda tag: tag.has_attr('class') and any(re.search(r'ingredients', class_name) for class_name in tag['class']))
    soup_str = str(ingredient_sections)
    clean_text = re.sub(r'<[^>]+>', '', soup_str).strip()
    
    # Split the ingredients into a list
    ingredients_list = clean_text.split('\n')
    
    return ingredients_list

def get_directions(soup):
    print("📜 Extracting the directions... Please wait.")
    progress_bar("Extracting directions")
    directions_sections = soup.find_all('ol')
    directions_steps = []
    for tag in directions_sections:
        if hasattr(tag, 'find_all'):    
            for li in tag.find_all('li'):
                directions = li.get_text().replace('\n', '').replace('\t', '').replace('\xa0', '').strip()
                step = re.sub(r"Serious Eats.*$", "", directions)
                directions_steps.append(step.rstrip())
    return directions_steps

def tools_used(soup):
    print("🔧 Analyzing tools used in the recipe... Please wait.")
    progress_bar("Analyzing tools")
    directions_steps = get_directions(soup)
    text = directions_steps
    tool_docs = [nlp(tool) for tool in cooking_tools]
    doc = nlp(' '.join(text))
    similarity_threshold = 0.6
    mentioned_tools = []
    for token in doc:
        for tool_doc in tool_docs:
            if token.similarity(tool_doc) > similarity_threshold:
                mentioned_tools.append(tool_doc.text)
                break  # Once a tool is matched, no need to compare with others
    
    mentioned_tools = list(set(mentioned_tools))
    return mentioned_tools

def help_search(query):
    print("❓ Press 1 for Google search, press 2 for YouTube search.")
    choice = input("Your choice: ").strip()
    search_query = query.replace(" ", "+")
    if choice == "1":
        print("🔍 Performing Google search... Please wait.")
        progress_bar("Google search")
        webbrowser.open(f"https://www.google.com/search?q={search_query}")
    elif choice == "2":
        print("🎥 Performing YouTube search... Please wait.")
        progress_bar("YouTube search")
        webbrowser.open(f"https://www.youtube.com/results?search_query={search_query}")
    else:
        print("🚫 Invalid choice, returning to the main menu.")

def chatbot():
    print("👋 Hello! Welcome to the Interactive Cookbook Chatbot!")
    url = input("Please enter the recipe URL: ").strip()
    
    driver = get_driver()
    soup = get_soup(driver, url)
    
    while True:
        print("\n📜 Menu:")
        print("1️⃣ Show Title")
        print("2️⃣ Show Ingredients")
        print("3️⃣ Show Directions")
        print("4️⃣ Show Tools Used")
        print("5️⃣ Help")
        print("6️⃣ End")
        
        choice = input("Please enter your choice (1-6): ").strip()
        
        if choice == "1":
            title = get_title(soup)
            print(title)
        
        elif choice == "2":
            ingredients = get_ingredients(soup)
            print("🍽 Ingredients:")
            for idx, ingredient in enumerate(ingredients, start=1):
                print(f"{idx}. {ingredient}")
        
        elif choice == "3":
            directions = get_directions(soup)
            print("📋 Directions:")
            for idx, step in enumerate(directions, start=1):
                print(f"Step {idx}: {step}")
        
        elif choice == "4":
            tools = tools_used(soup)
            print("🛠 Tools Used:")
            for idx, tool in enumerate(tools, start=1):
                print(f"{idx}. {tool}")
        
        elif choice == "5":
            query = input("Enter your query: ").strip()
            help_search(query)
        
        elif choice == "6":
            print("👋 Goodbye! Have a lovely day and happy cooking! 🍳")
            driver.quit()
            break
        
        else:
            print("🚫 Invalid choice. Please enter a number between 1 and 6.")

# Run the chatbot
chatbot()


👋 Hello! Welcome to the Interactive Cookbook Chatbot!


Please enter the recipe URL:  https://www.seriouseats.com/classic-banana-pudding-recipe-8648619


🌟 Setting up the web driver... Please wait.


Setting up driver: 100%|████████████████████████████████████████████████████████████████████████████████████ [🕒 00:00]


🔗 Processing the URL... Please wait.


...

🔍 Performing Google search... Please wait.


Google search: 100%|████████████████████████████████████████████████████████████████████████████████████████ [🕒 00:00]



📜 Menu:
1️⃣ Show Title
2️⃣ Show Ingredients
3️⃣ Show Directions
4️⃣ Show Tools Used
5️⃣ Help
6️⃣ End


Please enter your choice (1-6):  6


👋 Goodbye! Have a lovely day and happy cooking! 🍳
